## 导入工具包

In [1]:
from tools25 import * 

## 1, 登录
在tools25文件的login方法中填写用户名和密码后保存然后来到本文件Restart Kernal后重新import tools23

In [2]:
s = login()

b'<html>\r\n<head><title>504 Gateway Time-out</title></head>\r\n<body>\r\n<center><h1>504 Gateway Time-out</h1></center>\r\n</body>\r\n</html>\r\n'


## 2, 根据dataset_id，调取平台API获取datafields

In [15]:
df = get_datafields(s, dataset_id = 'fundamental6', region='USA', universe='TOP3000', delay=1)
print(df)

                  id                                   description  \
0             assets                                Assets - Total   
1        assets_curr                        Current Assets - Total   
2       bookvalue_ps                          Book Value Per Share   
3              capex                          Capital Expenditures   
4               cash                                          Cash   
..               ...                                           ...   
881            sales                          Sales/Turnover (Net)   
882     sales_growth                   Growth in Sales (Quarterly)   
883         sales_ps                   Sales per Share (Quarterly)   
884      sga_expense  Selling, General and Administrative Expenses   
885  working_capital               Working Capital (Balance Sheet)   

                                               dataset  \
0    {'id': 'fundamental6', 'name': 'Company Fundam...   
1    {'id': 'fundamental6', 'name': 'Compan

### Matrix Data

In [16]:
# print(df[df['type'] == "MATRIX"]["id"].tolist())
pc_fields = process_datafields_opt(df, "matrix")
print(pc_fields[:3])
print(len(pc_fields))

[]
0


## 3, Alpha factory（一阶Alpha模板，生成Matrix数据之后，开始跑这个)
### start with First Order

In [17]:
ops_lst=get_ops(s)['name'].to_list()    # 获取你账号可用的ops
ops=intersection_of_lists(ops_lst,ts_ops)
first_order = get_first_order(pc_fields, ops,prefix='') # 新增prefix,提效first order
print(first_order[:10])
print(len(first_order))

['ts_sum', 'ts_zscore', 'ts_std_dev', 'ts_mean', 'ts_scale', 'ts_rank', 'ts_quantile', 'ts_arg_min', 'ts_product', 'ts_arg_max', 'ts_delta']
[]
0


In [18]:
fo_alpha_dict = defaultdict(list)
init_decay = 5
regions = ["usa"]
for key in regions:
    for alpha in first_order:
        fo_alpha_dict[key].append((alpha,init_decay))

## 4, simulate alphas

In [19]:
# Simulate First Order
region_dict = {"usa" : ("USA", "TOP3000")}

first_layer_bag = []
simulate(fo_alpha_dict, region_dict, "opt_fo_usa", 'SUBINDUSTRY', 0 , first_layer_bag)

b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'
stones:
0


[]

## 5, Select alphas(根据时间取出刚刚simulate的alpha，用于跑二阶模板)
go to web alphas penal to look for the number and date to track for next order improve

In [20]:
## get promising alphas to improve in the next order
# 注意这里的时间是纽约时间，比北京时间慢13个小时
from tools25 import *


fo_tracker = get_alphas("04-28", "04-29", 1.0, 0.5, "USA", 320, "track")

b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'
0
['gavJ5J0', 'group_neutralize(group_neutralize(group_zscore(ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 120),densify(industry)),densify(pv13_h_min2_focused_pureplay_3000_sector)),densify(pv13_h_min2_3000_sector))', 2.52, 0.0493, 2.3, 0.004231, '2025-04-28T20:27:43-04:00', 5]
['r7k3l18', 'group_neutralize(group_neutralize(group_zscore(ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 120),densify(industry)),densify(pv13_r2_min2_3000_sector)),densify(pv13_h_min2_3000_sector))', 2.48, 0.0494, 2.25, 0.004176, '2025-04-28T20:56:52-04:00', 5]
['aaJY8xv', 'group_zscore(group_neutralize(group_zscore(ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 120),densify(industry)),densify(pv13_r2_min2_3000_sector)),densify(sector))', 2.47, 0.0461, 2.28, 0.004621, '2025

## 6, Next order improvement - Second Order(二阶模板，对一阶Alpha中性化处理)
second order: ts_ops(field, days) -> group_ops(ts_ops(field, days), group)

In [21]:
s=login()
print(len(fo_tracker['next']))
print(len(fo_tracker['decay']))

fo_layer = prune(fo_tracker['next'] + fo_tracker['decay'], 'usa', 'fnd6', 5)

ops_lst=get_ops(s)['name'].to_list()    # 获取你账号可用的ops
group_ops = ["group_neutralize", "group_rank", "group_normalize", "group_scale", "group_zscore"]
ops=intersection_of_lists(ops_lst,group_ops)
flst=get_fields_all()       # 获取你的可用字段

so_alpha_dict = defaultdict(list)
for key, couples in fo_layer.items():
    for expr, decay in couples:
        for alpha in get_group_second_order_factory([expr], ops, key,flst):
            so_alpha_dict[key].append((alpha,decay))


# print(so_alpha_dict['usa'][:3])
print(len(so_alpha_dict['usa']))


b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'
242
0
b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'
0
0
0
0
0
0
0
0
6272


### Simulate second order

In [ ]:
region_dict = {"usa" : ("USA", "TOP3000")}

so_bag = []
simulate(so_alpha_dict, region_dict, "opt_so_usa", 'SUBINDUSTRY', 0, so_bag)

b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'
curr usa len 6272
group 0 group_scale(group_neutralize(group_neutralize(group_zscore(ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 120),densify(industry)),densify(pv13_h_min2_focused_pureplay_3000_sector)),densify(pv13_h_min2_3000_sector)),densify(industry)) USA TOP3000 5
201
group 0 group_scale(group_neutralize(group_neutralize(group_zscore(ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 120),densify(industry)),densify(pv13_h_min2_focused_pureplay_3000_sector)),densify(pv13_h_min2_3000_sector)),densify(market)) USA TOP3000 5
201
group 0 group_scale(group_neutralize(group_neutralize(group_zscore(ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 120),densify(industry)),densify(pv13_h_min2_focused_pureplay_3000_sector)),densify(pv13_h_min2_3000_sector)),de

## Get submittable alphas（检查simulatied alphass是否可以submit）
check_submission and view alphas

In [3]:
# 输入你simulate运行日期，这里的时间都是纽约时间，比北京时间慢13小时
# 如果你是24号上午运行的simulate，那么这里的时间应该是23-24
import traceback


th_tracker = get_alphas("04-29", "04-30", 1.25, 1, "USA", 430, "submit")
stone_bag = []
for alpha in th_tracker['next'] + th_tracker['decay']:
    stone_bag.append(alpha[0])  # 取出其中的alpha_id
stone_bag_len=len(stone_bag)
print(fr"stone_bag len {stone_bag_len}")
gold_bag_all = []

for s_step in range(0, stone_bag_len, 10):
    e_step=s_step+10
    if e_step>=stone_bag_len:
        e_step=stone_bag_len-1

    print(fr"stone_bag s_step:{s_step},e_step:{e_step}")
    stone_bag_temp=stone_bag[s_step:e_step]


    gold_bag = []
    try:
        check_submission(stone_bag_temp, gold_bag, 0)
    except Exception as e:
        traceback.print_exc()
        print(fr"check_submission unknown error")   
        print(fr"time now:{get_timenow()}") 

    print("___________________________________________________________________________________________________")
    print(fr"time now:{get_timenow()}")
    gold_bag_all+=gold_bag
    print(gold_bag)

b'{"user":{"id":"PS30320"},"token":{"expiry":14400.0},"permissions":["TUTORIAL"]}'
0
['da8r9Zj', 'group_neutralize(group_neutralize(group_neutralize(group_zscore(ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 120),densify(industry)),densify(pv13_h_min2_focused_pureplay_3000_sector)),densify(pv13_h_min2_3000_sector)),densify(pv13_h_min2_focused_pureplay_3000_sector))', 2.65, 0.0494, 2.5, 0.004513, '2025-04-29T19:53:51-04:00', 5]
['XawAVqa', 'group_neutralize(group_neutralize(group_neutralize(group_zscore(ts_mean(winsorize(ts_backfill(implied_volatility_call_10/implied_volatility_mean_10, 120), std=4), 120),densify(industry)),densify(pv13_h_min2_focused_pureplay_3000_sector)),densify(pv13_h_min2_3000_sector)),densify(pv13_r2_min2_3000_sector))', 2.62, 0.0497, 2.46, 0.004427, '2025-04-29T19:54:01-04:00', 5]
['xvwLdjb', 'group_neutralize(group_neutralize(group_neutralize(group_zscore(ts_mean(winsorize(ts_backfill(implied_volatility_call_1

In [4]:
# 可以尝试submit的alpha_id
# https://platform.worldquantbrain.com/alpha/6lmK2XO
# 替换掉以上链接末尾的6lmK2XO就可以在网页打开
gold_bag_all

[]